In [1]:
import json
import os

In [2]:
dir = './raw_data'
files = [os.path.join(dir, f) for f in os.listdir(dir) if os.path.isfile(os.path.join(dir, f)) and 'dr_ntu' in f]
names = []

dr_ntu_keys = set()
for file in files:
    with open(file, 'r') as f:
        profile = json.load(f)
    
    dr_ntu_keys = dr_ntu_keys | profile.keys()

len(dr_ntu_keys)

20

In [3]:
dr_ntu_keys

{'articles',
 'bibliometrics',
 'biography',
 'book_chapters',
 'books',
 'conferences',
 'designation',
 'dr_ntu',
 'email',
 'full_name',
 'github',
 'google_scholar',
 'grants',
 'interests',
 'keywords',
 'name_card',
 'orcid',
 'other_websites',
 'scopus',
 'web_of_science'}

In [4]:
dr_ntu_information = [
    'full_name', 'email', 'designation',
    'biography', 'grants'
    'dr_ntu', 'google_scholar', 'github', 'orcid', 'scopus', 'web_of_science', 'other_websites'
]

In [5]:
dir = './raw_data'
files = [os.path.join(dir, f) for f in os.listdir(dir) if os.path.isfile(os.path.join(dir, f)) and 'google_scholar' in f and 'publication' not in f]
names = []

google_scholar_keys = set()
for file in files:
    with open(file, 'r') as f:
        profile = json.load(f)
    
    google_scholar_keys = google_scholar_keys | profile.keys()

len(google_scholar_keys)

4

In [6]:
google_scholar_keys

{'citation_statistics', 'co_authors', 'interests', 'publications'}

In [8]:
google_scholar_information = [
    'citation_statistics', 'interests'
]

In [9]:
dir = './raw_data'
dr_ntu_files = [os.path.join(dir, f) for f in os.listdir(dir) if os.path.isfile(os.path.join(dir, f)) and 'dr_ntu' in f]

output_dir = './processed_data'

for dr_ntu in dr_ntu_files:
    profile = {}
    with open(dr_ntu, 'r') as f:
        dr_ntu = json.load(f)
    name = dr_ntu['full_name']

    profile.update({k:v for k, v in dr_ntu.items() if k in dr_ntu_information})

    google_scholar_file = os.path.join(dir, f'google_scholar_{name.lower().replace(" ", "_")}.json')

    if os.path.exists(google_scholar_file):
        with open(google_scholar_file, 'r') as f:
            google_scholar = json.load(f)
        profile.update({k:v for k, v in google_scholar.items() if k in google_scholar_information})
    with open(os.path.join(output_dir, f'{name.lower().replace(" ", "_")}.json'), 'w') as f:
        json.dump(profile, f)